# Semantično jedro z integracijo strežnika OpenBnB MCP

Ta zvezek prikazuje, kako uporabljati Semantično jedro z dejanskim strežnikom OpenBnB MCP za iskanje pravih nastanitev na Airbnb z uporabo MCPStdioPlugin. Za dostop do LLM uporablja Microsoft Foundry. Za nastavitev vaših okoljskih spremenljivk lahko sledite [Setup Lesson](/00-course-setup/README.md)


## Uvozi potrebne pakete


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Ustvarjanje povezave MCP vtičnika

Povezali se bomo s [OpenBnB MCP strežnikom](https://github.com/openbnb-org/mcp-server-airbnb) z uporabo MCPStdioPlugin. Ta strežnik omogoča iskanje na Airbnb preko paketa @openbnb/mcp-server-airbnb.


## Ustvarjanje odjemalca

V tem vzorcu bomo za dostop do LLM uporabili Microsoft Foundry. Prepričajte se, da so vaše okoljske spremenljivke pravilno nastavljene.


## Konfiguracija okolja

Nastavite nastavitve Azure OpenAI. Prepričajte se, da imate nastavljene naslednje okoljske spremenljivke:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Razumevanje integracije OpenBnB MCP

Ta zvezek se poveže z **resničnim OpenBnB MCP strežnikom**, ki omogoča dejansko funkcionalnost iskanja na Airbnb.

### Kako deluje:

1. **MCPStdioPlugin**: Uporablja komunikacijo prek standardnega vnosa/izhoda s MCP strežnikom
2. **Prava NPM Povezava**: Prenese in zažene `@openbnb/mcp-server-airbnb` preko npx
3. **Živi podatki**: Vrača dejanske podatke o nepremičninah iz Airbnb API-jev
4. **Odkritje funkcij**: Agent samodejno odkrije razpoložljive funkcije od MCP strežnika

### Razpoložljive funkcije:

Strežnik OpenBnB MCP običajno omogoča:
- **search_listings** - Iskanje nepremičnin na Airbnb po lokaciji in kriterijih
- **get_listing_details** - Pridobitev podrobnih informacij o določenih nepremičninah
- **check_availability** - Preverjanje razpoložljivosti za določene datume
- **get_reviews** - Pridobivanje ocen za nepremičnine
- **get_host_info** - Pridobitev informacij o gostiteljih nepremičnin

### Pogoji:

- Na vašem sistemu mora biti nameščen **Node.js**
- **Povezava z internetom** za prenos MCP strežniškega paketa
- Na voljo mora biti **NPX** (prisoten z Node.js)

### Preizkus povezave:

MCP strežnik lahko ročno preizkusite z zagonom:
```bash
npx -y @openbnb/mcp-server-airbnb
```

To bo preneslo in zagnalo OpenBnB MCP strežnik, na katerega se nato Semantic Kernel poveže za resnične podatke Airbnb.


## Zagon agenta z OpenBnB MCP strežnikom

Zdaj bomo zagnali AI agenta, ki se poveže z OpenBnB MCP strežnikom za iskanje pravih nastanitev Airbnb v Stockholmu za 2 odrasla in 1 otroka. Po želji lahko spremenite seznam `user_inputs` za prilagoditev iskalnih kriterijev.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Povzetek
Čestitke! Uspešno ste zgradili AI agenta, ki se povezuje z iskanjem nastanitev v realnem svetu prek Model Context Protocol (MCP):

## Uporabljene tehnologije:
- Semantic Kernel - Za gradnjo inteligentnih agentov z Azure OpenAI
- Microsoft Foundry - Za zmožnosti LLM in zaključevanje pogovorov
- MCP (Model Context Protocol) - Za standardizirano integracijo orodij
- OpenBnB MCP Server - Za dejansko Airbnb iskanje
- Node.js/NPX - Za poganjanje zunanjega MCP strežnika

## Kaj ste se naučili:
- MCP integracija: Povezovanje Semantic Kernel agentov z zunanjimi MCP strežniki
- Dostop do podatkov v realnem času: Iskanje dejanskih nepremičnin na Airbnb preko živih API-jev
- Protokol komunikacije: Uporaba stdio komunikacije med agentom in MCP strežnikom
- Odkritje funkcij: Samodejno odkrivanje razpoložljivih funkcij iz MCP strežnikov
- Pretakanje odgovorov: Zajemanje in beleženje klicev funkcij v realnem času
- Prikaz HTML: Oblikovanje agentovih odgovorov s stiliziranimi tabelami in interaktivnimi prikazi

## Naslednji koraki:
- Integrirati dodatne MCP strežnike (vreme, leti, restavracije)
- Zgraditi sistem več agentov, ki združuje MCP in A2A protokole
- Ustvariti lastne MCP strežnike za vaše podatkovne vire
- Izvesti trajno pomnjenje pogovora med sejami
- Namestiti agenta v Azure Functions z orkestracijo MCP strežnika
- Dodati avtentikacijo uporabnikov in možnosti rezervacij


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:
Ta dokument je bil preveden z uporabo AI prevajalske storitve [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, upoštevajte, da avtomatizirani prevodi lahko vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku velja za avtoritativni vir. Za kritične informacije priporočamo strokovni človeški prevod. Ne odgovarjamo za kakršnekoli nesporazume ali napačne interpretacije, ki izhajajo iz uporabe tega prevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
